In [2]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

# ruta_monic = "C:\\Users\\monic\\ml_accident_bgta"
# sys.path.append(ruta_monic)

from src.config.setup import SetupConfig
from src.data_workflow.create_tables.create_tables import retrieve_all_table_data



import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import folium

**Para acceder a los datos lo podemos hacer usando la función:**

`retrieve_all_table_data(<table_name>)`

Parámetros:
- table_name: nombre de la tabla

Para acceder al nombre de la tabla, hacerlo a tráves de la clase SetupConfig:
- `SetupConfig.TABLE_ACCIDENTE`
- `SetupConfig.TABLE_ACTOR_VIAL`
- `SetupConfig.TABLE_CAUSA`
- `SetupConfig.TABLE_LESIONADO`
- `SetupConfig.TABLE_MUERTO`
- `SetupConfig.TABLE_VEHICULO`
- `SetupConfig.TABLE_VIA`

# Lectura base de datos

In [4]:
LESIONADO = retrieve_all_table_data(SetupConfig.TABLE_LESIONADO)
print(f'El dataset tiene {LESIONADO.shape[0]} filas y {LESIONADO.shape[1]} columnas ')

MUERTO = retrieve_all_table_data(SetupConfig.TABLE_MUERTO)
print(f'El dataset tiene {MUERTO.shape[0]} filas y {MUERTO.shape[1]} columnas ')

ACCIDENTE = retrieve_all_table_data(SetupConfig.TABLE_ACCIDENTE)
print(f'El dataset tiene {ACCIDENTE.shape[0]} filas y {ACCIDENTE.shape[1]} columnas ')

VEHICULO = retrieve_all_table_data(SetupConfig.TABLE_VEHICULO)
print(f'El dataset tiene {VEHICULO.shape[0]} filas y {VEHICULO.shape[1]} columnas ')

ACTORVIAL = retrieve_all_table_data(SetupConfig.TABLE_ACTOR_VIAL)
print(f'El dataset tiene {ACTORVIAL.shape[0]} filas y {ACTORVIAL.shape[1]} columnas ')

CAUSA = retrieve_all_table_data(SetupConfig.TABLE_CAUSA)
print(f'El dataset tiene {CAUSA.shape[0]} filas y {CAUSA.shape[1]} columnas ')

VIA = retrieve_all_table_data(SetupConfig.TABLE_VIA)
print(f'El dataset tiene {VIA.shape[0]} filas y {VIA.shape[1]} columnas ')


El dataset tiene 250182 filas y 16 columnas 
El dataset tiene 8106 filas y 18 columnas 
El dataset tiene 473883 filas y 18 columnas 
El dataset tiene 1443037 filas y 9 columnas 
El dataset tiene 1677376 filas y 14 columnas 
El dataset tiene 683654 filas y 10 columnas 
El dataset tiene 483952 filas y 19 columnas 


# Descriptiva univariada

## Accidente

In [ ]:
tabla = ACCIDENTE.groupby(['GRAVEDAD'])['FORMULARIO'].count().reset_index()
tabla

In [ ]:
# Crear el gráfico de pastel
plt.figure(figsize=(8, 6))
sns.set_palette("dark")
sns.set(style="whitegrid")
plt.pie(tabla["FORMULARIO"], labels=tabla["GRAVEDAD"], autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Título del gráfico
plt.title('Gravedad Siniestros')

# Mostrar el gráfico
plt.show()

In [ ]:
tabla = ACCIDENTE.groupby(['CLASE_ACC'])['FORMULARIO'].count().reset_index()
tabla

In [ ]:
# Crear el gráfico de pastel
plt.figure(figsize=(8, 6))
sns.set_palette("Set2")
sns.set(style="whitegrid")
plt.pie(tabla["FORMULARIO"], labels=tabla["CLASE_ACC"], autopct='%1.1f%%', startangle=140, textprops={'fontsize': 9})
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Título del gráfico
plt.title('Clase de Siniestro')

# Mostrar el gráfico
plt.show()

In [ ]:
tabla = ACCIDENTE.groupby(['LATITUD', 'LONGITUD'])['FORMULARIO'].count().reset_index()
tabla = tabla.sort_values(by="FORMULARIO", ascending=False)
tabla

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd

m = folium.Map(location=[tabla['LATITUD'].mean(), tabla['LONGITUD'].mean()], zoom_start=11)
# Convertir las coordenadas a una lista de listas
data = tabla[['LATITUD', 'LONGITUD', 'FORMULARIO']].values.tolist()

gradient = {0.2: 'blue',  0.6: 'yellow',  1: 'red'}
# Crear el mapa de calor
HeatMap(data, radius=10, max_zoom=13, gradient=gradient).add_to(m)

# Guardar el mapa
m.save('mapa_accidentes_heatmap.html')

In [ ]:
## NUMERO DE ACCIDENTES AÑO
tabla = ACCIDENTE.groupby(['ANO_OCURRENCIA_ACC'])['FORMULARIO'].count().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(tabla, x="ANO_OCURRENCIA_ACC", y="FORMULARIO")
plt.ylabel("Cantidad Siniestros")

In [ ]:
tabla = ACCIDENTE.groupby(['MES_OCURRENCIA_ACC'])['FORMULARIO'].count().reset_index()
orden_meses = pd.DataFrame({"MES":["ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO", "JULIO", "AGOSTO","SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"],
                            "MES_NUM": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})
orden_meses
# Ordenar el DataFrame por el valor numérico de los meses
tabla = pd.merge(tabla, orden_meses, left_on="MES_OCURRENCIA_ACC", right_on="MES", how = "left")
tabla = tabla.sort_values(by='MES_NUM').reset_index(drop=True)
tabla



In [ ]:
# Gráfico
plt.figure(figsize=(13, 6))
sns.barplot(tabla, x="MES_OCURRENCIA_ACC", y="FORMULARIO")
plt.ylabel("Cantidad Siniestros")

In [ ]:
tabla = ACCIDENTE.groupby(['DIA_OCURRENCIA_ACC'])['FORMULARIO'].count().reset_index()
orden_dias = pd.DataFrame({"DIA":["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO", "DOMINGO"],
                            "DIA_NUM": [1, 2, 3, 4, 5, 6, 7]})
orden_meses
# Ordenar el DataFrame por el valor numérico de los meses
tabla = pd.merge(tabla, orden_dias, left_on="DIA_OCURRENCIA_ACC", right_on="DIA", how = "left")
tabla = tabla.sort_values(by='DIA_NUM').reset_index(drop=True)
tabla

In [ ]:
# Gráfico
plt.figure(figsize=(13, 6))
sns.barplot(tabla, x="DIA_OCURRENCIA_ACC", y="FORMULARIO")
plt.ylabel("Cantidad Siniestros")

In [ ]:
tabla = ACCIDENTE.groupby(['HORA_OCURRENCIA_ACC'])['FORMULARIO'].count().reset_index()
tabla['HORA_OCURRENCIA_ACC'] = pd.to_datetime(tabla['HORA_OCURRENCIA_ACC'])
tabla_grouped = tabla.groupby(tabla['HORA_OCURRENCIA_ACC'].dt.strftime('%H:00:00'))['FORMULARIO'].sum().reset_index()
tabla_grouped

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(tabla_grouped['HORA_OCURRENCIA_ACC'], tabla_grouped['FORMULARIO'])
plt.title('Siniestros por hora')
plt.xlabel('Hora de ocurrencia')
plt.ylabel('Cantidad Siniestros')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Actor Vial

In [ ]:
ActorVial.head(5)

In [ ]:
#ActorVial[ActorVial["Formulario"] == "A000239725"]
tabla = ActorVial.groupby(['FORMULARIO']).agg("size").reset_index(name = "conteo")
tabla = tabla.sort_values(by = "conteo", ascending=False)

# Definir los intervalos
bins = [0, 20, 40, 60, 80]

# Crear los intervalos y contar la frecuencia
conteo_intervalos = pd.cut(tabla['conteo'], bins=bins).value_counts().sort_index()
conteo_intervalos

In [ ]:
tabla.describe()

In [ ]:
tabla = ActorVial.groupby(['CONDICION']).agg("size").reset_index(name = "conteo")
# Gráfico
plt.figure(figsize=(9, 4))
sns.barplot(tabla, x='CONDICION', y="conteo")
plt.ylabel("Cantidad Accidentados")
plt.ticklabel_format(style='plain', axis='y')

for index, row in tabla.iterrows():
    plt.text(index, row.conteo, f"{row.conteo:,}", color='black', ha="center")
    
plt.show()

In [ ]:
tabla = ActorVial.groupby(['Estado'.upper()]).agg("size").reset_index(name = "conteo")
print(tabla)
# Gráfico
plt.figure(figsize=(9, 4))
sns.barplot(tabla, x="Estado".upper(), y="conteo")
plt.ylabel("Cantidad Accidentados")
plt.ticklabel_format(style='plain', axis='y')

for index, row in tabla.iterrows():
    plt.text(index, row.conteo, f"{row.conteo:,}", color='black', ha="center")
    
plt.show()

In [ ]:
tabla = ActorVial.groupby(['MUERTE_POSTERIOR']).agg("size").reset_index(name = "conteo")
# Crear el gráfico de pastel
plt.figure(figsize=(8, 6))
sns.set_palette("dark")
sns.set(style="whitegrid")
plt.pie(tabla["conteo"], labels=tabla["MUERTE_POSTERIOR"], autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Título del gráfico
plt.title('Muerte Posterior')

# Mostrar el gráfico
plt.show()

In [ ]:
tabla = ActorVial.groupby(['GENERO']).agg("size").reset_index(name = "conteo")
print(tabla)
# Gráfico
plt.figure(figsize=(9, 4))
sns.barplot(tabla, x="GENERO", y="conteo")
plt.ylabel("Genero Accidentados")
plt.ticklabel_format(style='plain', axis='y')

for index, row in tabla.iterrows():
    plt.text(index, row.conteo, f"{row.conteo:,}", color='black', ha="center")
    
plt.show()

In [ ]:
ActorVial["EDAD"].describe().round(0)

# Seleccion variables para cruce semana 6

## Union Lesionado y Muerto

Se une la tabla lesionado y muerto, identificando con variable indicadora, si fue lesionado o muerto <br>
Se omite 37 registros que no tienen "CLASE_ACC" <br>
Se omite 1 registro que no tiene "CONDICION" <br>
Se omite 64399 registros que no tiene codigo vehiculo <br>		
Cant data inicial: 258288, Cant data final = 193862. Prueba inicial 75% de los datos iniciales. <br>


In [5]:
LESIONADO = LESIONADO.drop(["OBJECTID"], axis=1)
LESIONADO["ESTADO_ACCIDENTADO"] = "LESIONADO"

MUERTO = MUERTO.drop(["OBJECTID", "MUERTE_POSTERIOR", "FECHA_POSTERIOR_MUERTE"], axis=1)
MUERTO["ESTADO_ACCIDENTADO"] = "MUERTO"

ESTADO_ACCIDENTADO = pd.concat([LESIONADO, MUERTO], axis=0, ignore_index=True).sort_values(by="FORMULARIO")
print(ESTADO_ACCIDENTADO.shape, 
      "\n NA CLASE_ACC", sum(ESTADO_ACCIDENTADO["CLASE_ACC"].isna()),
      "\n NA CONDICION", sum(ESTADO_ACCIDENTADO["CONDICION"].isna()), 
      "\n NA CODIGO_VEHICULO", sum(ESTADO_ACCIDENTADO["CODIGO_VEHICULO"].isna())
      )
ESTADO_ACCIDENTADO = ESTADO_ACCIDENTADO[~ESTADO_ACCIDENTADO["CLASE_ACC"].isna()]
ESTADO_ACCIDENTADO = ESTADO_ACCIDENTADO[~ESTADO_ACCIDENTADO["CONDICION"].isna()]
ESTADO_ACCIDENTADO = ESTADO_ACCIDENTADO[~ESTADO_ACCIDENTADO["CODIGO_VEHICULO"].isna()]
ESTADO_ACCIDENTADO["CODIGO_VEHICULO"] = ESTADO_ACCIDENTADO["CODIGO_VEHICULO"].astype("Int64")

print(ESTADO_ACCIDENTADO.shape)

(258288, 16) 
 NA CLASE_ACC 37 
 NA CONDICION 1 
 NA CODIGO_VEHICULO 64399
(193862, 16)


## Accidente
Usado para llevar a la tabla anterior (union lesionado muerto) el codigo del accidente para construir el codigo

In [6]:
ACCIDENTE['CODIGO_ACCIDENTE'] = ACCIDENTE['CODIGO_ACCIDENTE'].astype(str)
ACCIDENTE = ACCIDENTE[["FORMULARIO", "CODIGO_ACCIDENTE"]]

## Vehiculo

El codigo de vehiculo no asocia un tipo de vehiculo único. <br>
En dos formularios 1 puede ser automovil y en otro 1 puede ser volqueta.
Usado para identificar la clase de vehiculo implicado en el accidente por Codigo


In [9]:
VEHICULO["CODIGO_VEHICULO"] = VEHICULO["CODIGO_VEHICULO"].astype("Int64").astype("str")
VEHICULO = VEHICULO[["CODIGO", "CLASE", "SERVICIO"]]

## Actor Vial
Decido no usar: toda la informacion se encuentra en la union lesionado y muerto

## Causa
Se excluye los registros que no tienen causa del accidente <br>
Se concatena las diferentes causas en un codigo, para que quede unico el codigo y llevarle la causa al vehiculo culpable

In [11]:
CAUSA = CAUSA[~CAUSA["NOMBRE"].isna()]
CAUSA = CAUSA.groupby(["CODIGO_AC_VH"])['NOMBRE'].apply(lambda x: ', '.join(x)).reset_index()
CAUSA.rename(columns={"CODIGO_AC_VH": "CODIGO"}, inplace=True)

In [15]:
CAUSA.head()

,CODIGO,NOMBRE
0,1000-1,NO MANTENER DISTANCIA DE SEGURIDAD
1,10000-1,OTRA
2,10001-1,NO MANTENER DISTANCIA DE SEGURIDAD
3,10001-2,OTRA
4,10002-1,OTRA


## Via
Se omite 4713 registros que no tienen Estado de la via

## Construccion sabana

In [23]:
print(ESTADO_ACCIDENTADO.shape)

#Primer cruce
DF_ALL = pd.merge(ESTADO_ACCIDENTADO, ACCIDENTE, on = "FORMULARIO", how="left")
DF_ALL["CODIGO"] = DF_ALL["CODIGO_ACCIDENTE"]+"-"+ DF_ALL["CODIGO_VEHICULO"].astype("str")
print(DF_ALL.shape)

#Segundo cruce
DF_ALL = pd.merge(DF_ALL, VEHICULO, on = "CODIGO", how = "left")
print(DF_ALL.shape)

#Tercer cruce
DF_ALL = pd.merge(DF_ALL, CAUSA, on = "CODIGO", how = "left")
print(DF_ALL.shape)

(193862, 16)
(193862, 18)
(193862, 20)
(193862, 21)
